In [6]:
!pip install pandas


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 23.2.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
import os
import shutil
import random
import pandas as pd

# Paths
original_dataset = "PlantVillage"  # the folder you extracted
base_dir = "dataset"
train_dir = os.path.join(base_dir, "train")
test_dir = os.path.join(base_dir, "test")

# Create directories if not exist
for split in [train_dir, test_dir]:
    os.makedirs(split, exist_ok=True)

# To store CSV data
train_data = []
test_data = []

# Loop through each class (disease/healthy type)
for class_folder in os.listdir(original_dataset):
    class_path = os.path.join(original_dataset, class_folder)
    if not os.path.isdir(class_path):
        continue

    # Create subfolders in train and test
    os.makedirs(os.path.join(train_dir, class_folder), exist_ok=True)
    os.makedirs(os.path.join(test_dir, class_folder), exist_ok=True)

    # List images
    images = os.listdir(class_path)
    random.shuffle(images)

    # 80/20 split
    split_index = int(0.8 * len(images))
    train_files = images[:split_index]
    test_files = images[split_index:]

    # Move + add to CSV list
    for file in train_files:
        src = os.path.join(class_path, file)
        dst = os.path.join(train_dir, class_folder, file)
        shutil.copy(src, dst)
        train_data.append([dst, class_folder])  # filepath, label

    for file in test_files:
        src = os.path.join(class_path, file)
        dst = os.path.join(test_dir, class_folder, file)
        shutil.copy(src, dst)
        test_data.append([dst, class_folder])  # filepath, label

# Convert to DataFrame
train_df = pd.DataFrame(train_data, columns=["filepath", "label"])
test_df = pd.DataFrame(test_data, columns=["filepath", "label"])

# Save CSV files
train_df.to_csv("train_labels.csv", index=False)
test_df.to_csv("test_labels.csv", index=False)

print("✅ Dataset split and CSV files created successfully!")


✅ Dataset split and CSV files created successfully!


In [8]:
!pip install tensorflow pillow matplotlib scikit-learn --quiet



[notice] A new release of pip is available: 23.2.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
import os
import json
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input

SEED = 42
tf.random.set_seed(SEED)
np.random.seed(SEED)

BASE_DIR = "dataset"   # contains train/ and test/
TRAIN_CSV = "train_labels.csv"
TEST_CSV  = "test_labels.csv"

IMG_SIZE = (224, 224)
BATCH_SIZE = 32


In [10]:
train_df = pd.read_csv(TRAIN_CSV)
test_df  = pd.read_csv(TEST_CSV)

# your CSV has columns: filepath, label
train_df["filepath"] = train_df["filepath"].apply(lambda p: os.path.abspath(p))
test_df["filepath"]  = test_df["filepath"].apply(lambda p: os.path.abspath(p))

# quick sanity checks
print("Train rows:", len(train_df), " | Test rows:", len(test_df))
print("Classes (train):", train_df["label"].nunique())
print(train_df["label"].value_counts().head())


Train rows: 16505  | Test rows: 4134
Classes (train): 15
label
Tomato__Tomato_YellowLeaf__Curl_Virus          2567
Tomato_Bacterial_spot                          1701
Tomato_Late_blight                             1527
Tomato_Septoria_leaf_spot                      1416
Tomato_Spider_mites_Two_spotted_spider_mite    1340
Name: count, dtype: int64


In [11]:
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    validation_split=0.2,          # 20% of train_df becomes validation
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True
)

valid_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    validation_split=0.2
)

test_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input
)

train_gen = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col="filepath",
    y_col="label",
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    subset="training",
    shuffle=True,
    seed=SEED
)

val_gen = valid_datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col="filepath",
    y_col="label",
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    subset="validation",
    shuffle=False
)

test_gen = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    x_col="filepath",
    y_col="label",
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=False
)

num_classes = train_gen.num_classes
class_indices = train_gen.class_indices
index_to_label = {v:k for k,v in class_indices.items()}
print("Classes:", index_to_label)


Found 13204 validated image filenames belonging to 15 classes.
Found 3301 validated image filenames belonging to 15 classes.
Found 4133 validated image filenames belonging to 15 classes.


C:\Users\perum\AppData\Roaming\Python\Python311\site-packages\keras\src\legacy\preprocessing\image.py:920: UserWarning: Found 1 invalid image filename(s) in x_col="filepath". These filename(s) will be ignored.
  warnings.warn(


AttributeError: 'DataFrameIterator' object has no attribute 'num_classes'

In [12]:
import os, numpy as np, pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

SEED = 42
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

# 1) Load CSVs
train_df = pd.read_csv("train_labels.csv")
test_df  = pd.read_csv("test_labels.csv")

# 2) Make absolute paths (Windows-safe) + ensure strings
train_df["filepath"] = train_df["filepath"].astype(str).apply(os.path.abspath)
test_df["filepath"]  = test_df["filepath"].astype(str).apply(os.path.abspath)
train_df["label"] = train_df["label"].astype(str)
test_df["label"]  = test_df["label"].astype(str)

# 3) Drop bad / missing / non-image files
valid_ext = {".jpg", ".jpeg", ".png", ".bmp", ".gif"}
def is_good(p):
    return isinstance(p, str) and os.path.exists(p) and os.path.splitext(p)[1].lower() in valid_ext

bad_train = train_df[~train_df["filepath"].apply(is_good)]
bad_test  = test_df[~test_df["filepath"].apply(is_good)]
if not bad_train.empty or not bad_test.empty:
    print("Removing bad rows:\n",
          f"- train: {len(bad_train)}\n- test:  {len(bad_test)}")
train_df = train_df[train_df["filepath"].apply(is_good)].drop_duplicates("filepath")
test_df  = test_df[test_df["filepath"].apply(is_good)].drop_duplicates("filepath")

print("After cleaning → train:", len(train_df), " test:", len(test_df))

# 4) Build generators (with internal validation split from train_df)
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    validation_split=0.20,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True
)
valid_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    validation_split=0.20
)
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_gen = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col="filepath",
    y_col="label",
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    subset="training",
    shuffle=True,
    seed=SEED
)
val_gen = valid_datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col="filepath",
    y_col="label",
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    subset="validation",
    shuffle=False
)
test_gen = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    x_col="filepath",
    y_col="label",
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=False
)

# 5) FIX: derive num_classes from class_indices (works in Keras 3 legacy)
class_indices = train_gen.class_indices
num_classes = len(class_indices)
index_to_label = {v: k for k, v in class_indices.items()}

print("Classes:", index_to_label)
print("num_classes:", num_classes)


Removing bad rows:
 - train: 0
- test:  1
After cleaning → train: 16505  test: 4133
Found 13204 validated image filenames belonging to 15 classes.
Found 3301 validated image filenames belonging to 15 classes.
Found 4133 validated image filenames belonging to 15 classes.
Classes: {0: 'Pepper__bell___Bacterial_spot', 1: 'Pepper__bell___healthy', 2: 'Potato___Early_blight', 3: 'Potato___Late_blight', 4: 'Potato___healthy', 5: 'Tomato_Bacterial_spot', 6: 'Tomato_Early_blight', 7: 'Tomato_Late_blight', 8: 'Tomato_Leaf_Mold', 9: 'Tomato_Septoria_leaf_spot', 10: 'Tomato_Spider_mites_Two_spotted_spider_mite', 11: 'Tomato__Target_Spot', 12: 'Tomato__Tomato_YellowLeaf__Curl_Virus', 13: 'Tomato__Tomato_mosaic_virus', 14: 'Tomato_healthy'}
num_classes: 15


In [13]:
labels_array = train_df["label"].values
class_weights = compute_class_weight(
    class_weight="balanced",
    classes=np.unique(labels_array),
    y=labels_array
)
# map from class index (0..num_classes-1) to weight
class_weight_dict = {class_indices[c]: w for c, w in zip(np.unique(labels_array), class_weights)}
class_weight_dict


{0: np.float64(1.3805938937682978),
 1: np.float64(0.9309080654258319),
 2: np.float64(1.3754166666666667),
 3: np.float64(1.3754166666666667),
 4: np.float64(9.09366391184573),
 5: np.float64(0.6468743876151284),
 6: np.float64(1.3754166666666667),
 7: np.float64(0.7205850251036892),
 8: np.float64(1.445904511607534),
 9: np.float64(0.7770715630885122),
 10: np.float64(0.8211442786069652),
 11: np.float64(0.9798159691303058),
 12: np.float64(0.4286456304376055),
 13: np.float64(3.692393736017897),
 14: np.float64(0.865041928721174)}

In [14]:
base_model = MobileNetV2(
    input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3),
    include_top=False,
    weights="imagenet"
)
base_model.trainable = False  # freeze for initial training

inputs = layers.Input(shape=(IMG_SIZE[0], IMG_SIZE[1], 3))
x = base_model(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(num_classes, activation="softmax")(x)
model = models.Model(inputs, outputs)

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 15)             │        19,215 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,277,199 (8.69 MB)

 Trainable params: 19,215 (75.06 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [16]:
callbacks = [
    EarlyStopping(patience=3, restore_best_weights=True, monitor="val_accuracy"),
    ModelCheckpoint("leaf_disease_model.keras", save_best_only=True, monitor="val_accuracy")
]

history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=10,
    callbacks=callbacks
)


Epoch 1/10
413/413 ━━━━━━━━━━━━━━━━━━━━ 594s 1s/step - accuracy: 0.7122 - loss: 0.8801 - val_accuracy: 0.1039 - val_loss: 9.4209
Epoch 2/10
413/413 ━━━━━━━━━━━━━━━━━━━━ 763s 2s/step - accuracy: 0.8304 - loss: 0.5181 - val_accuracy: 0.1109 - val_loss: 10.2544
Epoch 3/10
413/413 ━━━━━━━━━━━━━━━━━━━━ 682s 2s/step - accuracy: 0.8501 - loss: 0.4478 - val_accuracy: 0.1121 - val_loss: 10.8347
Epoch 4/10
413/413 ━━━━━━━━━━━━━━━━━━━━ 1382s 3s/step - accuracy: 0.8610 - loss: 0.4150 - val_accuracy: 0.1094 - val_loss: 11.2011
Epoch 5/10
413/413 ━━━━━━━━━━━━━━━━━━━━ 650s 2s/step - accuracy: 0.8664 - loss: 0.3910 - val_accuracy: 0.1154 - val_loss: 11.8385
Epoch 6/10
413/413 ━━━━━━━━━━━━━━━━━━━━ 720s 2s/step - accuracy: 0.8682 - loss: 0.3867 - val_accuracy: 0.1148 - val_loss: 11.9365
Epoch 7/10
413/413 ━━━━━━━━━━━━━━━━━━━━ 699s 2s/step - accuracy: 0.8719 - loss: 0.3731 - val_accuracy: 0.1112 - val_loss: 12.2799
Epoch 8/10
413/413 ━━━━━━━━━━━━━━━━━━━━ 904s 2s/step - accuracy: 0.8761 - loss: 0.3592 - v

In [ ]:
# unfreeze last ~30 layers of the base model for fine-tuning
for layer in base_model.layers[-30:]:
    layer.trainable = True

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

history_ft = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=5,
    callbacks=callbacks,
    class_weight=class_weight_dict  # remove if not using class weights
)


NameError: name 'base_model' is not defined

In [17]:
test_loss, test_acc = model.evaluate(test_gen)
print(f"Test accuracy: {test_acc*100:.2f}%")


130/130 ━━━━━━━━━━━━━━━━━━━━ 146s 1s/step - accuracy: 0.7293 - loss: 2.6337
Test accuracy: 72.93%


In [18]:
# unfreeze last ~30 layers of the base model for fine-tuning
for layer in base_model.layers[-30:]:
    layer.trainable = True

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

history_ft = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=5,
    callbacks=callbacks,
    class_weight=class_weight_dict  # remove if not using class weights
)


Epoch 1/5
413/413 ━━━━━━━━━━━━━━━━━━━━ 835s 2s/step - accuracy: 0.8623 - loss: 0.4022 - val_accuracy: 0.0827 - val_loss: 17.0355
Epoch 2/5
413/413 ━━━━━━━━━━━━━━━━━━━━ 889s 2s/step - accuracy: 0.9293 - loss: 0.1847 - val_accuracy: 0.1030 - val_loss: 19.8397
Epoch 3/5
413/413 ━━━━━━━━━━━━━━━━━━━━ 680s 2s/step - accuracy: 0.9504 - loss: 0.1229 - val_accuracy: 0.1066 - val_loss: 19.8369


In [19]:
test_loss, test_acc = model.evaluate(test_gen)
print(f"Test accuracy: {test_acc*100:.2f}%")

130/130 ━━━━━━━━━━━━━━━━━━━━ 131s 1s/step - accuracy: 0.6731 - loss: 3.8785
Test accuracy: 67.31%


In [20]:
from tensorflow.keras.models import load_model
import json

# Load the saved head-only model
model = load_model("leaf_disease_model.keras")

# Reload label map
with open("label_map.json", "r") as f:
    index_to_label = json.load(f)

print("Model and label map reloaded ✅")


FileNotFoundError: [Errno 2] No such file or directory: 'label_map.json'

In [21]:
import json

# Make sure train_gen is already defined from earlier cells
class_indices = train_gen.class_indices
index_to_label = {v: k for k, v in class_indices.items()}

with open("label_map.json", "w") as f:
    json.dump(index_to_label, f, indent=2)

print("Saved: label_map.json")


Saved: label_map.json


In [22]:
from tensorflow.keras.models import load_model
import json

model = load_model("leaf_disease_model.keras")

with open("label_map.json", "r") as f:
    index_to_label = json.load(f)

print("Model and label map reloaded ✅")


Model and label map reloaded ✅


In [23]:
test_loss, test_acc = model.evaluate(test_gen)
print(f"Test accuracy (reloaded model): {test_acc*100:.2f}%")


130/130 ━━━━━━━━━━━━━━━━━━━━ 128s 953ms/step - accuracy: 0.7293 - loss: 2.6337
Test accuracy (reloaded model): 72.93%


In [24]:
# model file already saved by ModelCheckpoint as leaf_disease_model.keras
with open("label_map.json", "w") as f:
    json.dump(index_to_label, f, indent=2)

print("Saved: leaf_disease_model.keras and label_map.json")


Saved: leaf_disease_model.keras and label_map.json


In [25]:
from tensorflow.keras.utils import load_img, img_to_array

def predict_image(path):
    img = load_img(path, target_size=IMG_SIZE)
    arr = img_to_array(img)
    arr = np.expand_dims(arr, axis=0)
    arr = preprocess_input(arr)
    probs = model.predict(arr)
    idx = np.argmax(probs[0])
    return index_to_label[idx], float(probs[0][idx])

# Try it on one test image
sample_path = test_df.iloc[0]["filepath"]
pred, p = predict_image(sample_path)
print("Prediction:", pred, "| confidence:", round(p, 3))
sample_path


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step


KeyError: np.int64(6)

In [26]:
# model file already saved by ModelCheckpoint as leaf_disease_model.keras

# ensure keys are integers before saving
index_to_label_int = {int(v): k for v, k in index_to_label.items()}

with open("label_map.json", "w") as f:
    json.dump(index_to_label_int, f, indent=2)

print("Saved: leaf_disease_model.keras and label_map.json")


Saved: leaf_disease_model.keras and label_map.json


In [27]:
from tensorflow.keras.utils import load_img, img_to_array
import numpy as np

def predict_image(path):
    img = load_img(path, target_size=IMG_SIZE)
    arr = img_to_array(img)
    arr = np.expand_dims(arr, axis=0)
    arr = preprocess_input(arr)
    probs = model.predict(arr)
    idx = np.argmax(probs[0])
    return index_to_label[idx], float(probs[0][idx])

# Try again
sample_path = test_df.iloc[0]["filepath"]
pred, p = predict_image(sample_path)
print("Prediction:", pred, "| confidence:", round(p, 3))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step


KeyError: np.int64(6)

In [28]:
from tensorflow.keras.models import load_model
import json

model = load_model("leaf_disease_model.keras")

with open("label_map.json", "r") as f:
    index_to_label = json.load(f)   # already has integer keys


In [29]:
from tensorflow.keras.utils import load_img, img_to_array
import numpy as np

def predict_image(path):
    img = load_img(path, target_size=IMG_SIZE)
    arr = img_to_array(img)
    arr = np.expand_dims(arr, axis=0)
    arr = preprocess_input(arr)
    probs = model.predict(arr)
    idx = np.argmax(probs[0])
    return index_to_label[idx], float(probs[0][idx])

# Try again
sample_path = test_df.iloc[0]["filepath"]
pred, p = predict_image(sample_path)
print("Prediction:", pred, "| confidence:", round(p, 3))


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step


KeyError: np.int64(6)

In [4]:
from tensorflow.keras.utils import load_img, img_to_array
import numpy as np

# Ensure label map keys are integers (important for lookups)
index_to_label = {int(k): v for k, v in index_to_label.items()}

def predict_image(path):
    img = load_img(path, target_size=IMG_SIZE)   # load image
    arr = img_to_array(img)                      # convert to array
    arr = np.expand_dims(arr, axis=0)            # add batch dimension
    arr = preprocess_input(arr)                  # preprocess for MobileNetV2
    probs = model.predict(arr)                   # model prediction
    idx = np.argmax(probs[0])                    # predicted class index
    return index_to_label[idx], float(probs[0][idx])

# Try it on one test image
sample_path = test_df.iloc[0]["filepath"]
pred, p = predict_image(sample_path)
print("Prediction:", pred, "| confidence:", round(p, 3))
sample_path


NameError: name 'index_to_label' is not defined

In [3]:
import random

# Pick 5 random test images
sample_paths = random.sample(list(test_df["filepath"]), 5)

for path in sample_paths:
    pred, p = predict_image(path)
    print(f"File: {os.path.basename(path)}")
    print(f" → Prediction: {pred} | Confidence: {round(p, 3)}")
    print("-"*50)


NameError: name 'test_df' is not defined

In [2]:
# Evaluate on training data
train_loss, train_acc = model.evaluate(X_train, y_train, verbose=0)
print(f"Train Accuracy: {train_acc:.4f}, Train Loss: {train_loss:.4f}")

# Evaluate on test/validation data
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Accuracy: {test_acc:.4f}, Test Loss: {test_loss:.4f}")


NameError: name 'model' is not defined

In [5]:
from tensorflow.keras.models import load_model

model = load_model("plant_disease_model.keras")   # load your trained model


C:\Users\perum\AppData\Roaming\Python\Python311\site-packages\keras\src\saving\saving_lib.py:797: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 36 variables whereas the saved optimizer has 70 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [6]:
IMG_SIZE = (224, 224)   # MobileNetV2 default


In [9]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMG_SIZE = (224, 224)

datagen = ImageDataGenerator(preprocessing_function=None)  # same preprocessing as training
train_gen_tmp = datagen.flow_from_directory(
    "dataset",   # 👈 your training dataset folder
    target_size=IMG_SIZE,
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

class_indices = train_gen_tmp.class_indices
index_to_label = {v: k for k, v in class_indices.items()}


Found 27298 images belonging to 2 classes.


In [11]:
print(train_gen_tmp.class_indices)


{'test': 0, 'train': 1}


In [12]:
index_to_label = {v: k for k, v in train_gen_tmp.class_indices.items()}
print(index_to_label)   


{0: 'test', 1: 'train'}


In [18]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMG_SIZE = (224, 224)  # must match training size

test_datagen = ImageDataGenerator(
    preprocessing_function=None  # use the same preprocessing as training if needed
)

test_generator = test_datagen.flow_from_directory(
    "dataset/test",   # 👈 replace with your test folder path
    target_size=IMG_SIZE,
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)


Found 7463 images belonging to 15 classes.


In [19]:
loss, acc = model.evaluate(test_generator, verbose=1)
print(f"Test Accuracy: {acc:.4f}")
print(f"Test Loss: {loss:.4f}")


234/234 ━━━━━━━━━━━━━━━━━━━━ 1711s 7s/step - accuracy: 0.0722 - loss: 6.5113
Test Accuracy: 0.0722
Test Loss: 6.5113


In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# 1. Load your fine-tuned model
model = load_model("plant_disease_model.keras")

# 2. Use the SAME preprocessing as in Colab
test_datagen = ImageDataGenerator(rescale=1./255)

# 3. Point this to your test dataset folder
test_dir = "dataset/test"   # <-- change this to your test folder path

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),   # must match training image_size
    batch_size=32,
    class_mode="categorical",
    shuffle=False             # important for evaluation
)

# 4. Evaluate the model
loss, accuracy = model.evaluate(test_generator)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")


C:\Users\perum\AppData\Roaming\Python\Python311\site-packages\keras\src\saving\saving_lib.py:797: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 36 variables whereas the saved optimizer has 70 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Found 7463 images belonging to 15 classes.


C:\Users\perum\AppData\Roaming\Python\Python311\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


234/234 ━━━━━━━━━━━━━━━━━━━━ 1552s 7s/step - accuracy: 0.9433 - loss: 0.1673
Test Loss: 0.1673
Test Accuracy: 0.9433


In [3]:
loss, acc = model.evaluate(test_generator)
print(f"Test Loss: {loss:.4f}, Test Accuracy: {acc:.4f}")

# Save metrics
import json
with open("metrics.json", "w") as f:
    json.dump({"loss": float(loss), "accuracy": float(acc)}, f)


234/234 ━━━━━━━━━━━━━━━━━━━━ 1187s 5s/step - accuracy: 0.9433 - loss: 0.1673
Test Loss: 0.1673, Test Accuracy: 0.9433
